In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
import jax
import jax.numpy as jnp
from jax import random

import optax
from numpyro.infer import SVI, Trace_ELBO, TraceEnum_ELBO

In [23]:
from src.models.M2VAE import M2VAE

from src.models.encoder_decoder import MNISTEncoder, MNISTDecoder
from src.data_loading.constants import MNIST_IMG_SHAPE

from src.data_loading.loaders import get_data_loaders

# Set Random Seed

In [24]:
seed = 42

# Load Data

In [25]:
img_shape, loader_dict, size_dict = get_data_loaders(dataset_name="MNIST", 
                                          p_test=0.2, 
                                          p_val=0.2, 
                                          p_supervised=0.1, 
                                          batch_size=2, 
                                          num_workers=0, 
                                          seed=seed)

scale_factor = 50 / size_dict["supervised"]

Successfully loaded MNIST dataset.
Total num samples 60000
Num test samples: 12000
Num validation samples: 9600
Num supervised samples: 3840
Num unsupervised samples: 34560


# Set up Model

In [26]:
m2_vae = M2VAE(MNISTEncoder, MNISTDecoder, 10, 30, MNIST_IMG_SHAPE, scale_factor=scale_factor)

# Set up Optimizer

In [27]:
optimizer = optax.adam(1e-3)

# Set up Statistic Variational Inference for all cases (Supervised, Unsupervised, and Classify)

In [28]:
svi_supervised = SVI(m2_vae.model_supervised, 
            m2_vae.guide_supervised, 
            optim=optimizer, 
            loss=Trace_ELBO()
)

svi_unsupervised = SVI(m2_vae.model_unsupervised, 
            m2_vae.guide_unsupervised, #config_enumerate(m2_vae.guide_unsupervised), 
            optim=optimizer, 
            loss=Trace_ELBO() # TraceEnum_ELBO(max_plate_nesting=1) Would be better, ...
)

svi_classify = SVI(m2_vae.model_classify,
                   m2_vae.guide_classify,
                   optim=optimizer,
                   loss=Trace_ELBO()
)

In [29]:
state = svi_supervised.init(random.PRNGKey(0), xs=jnp.ones((1,)+MNIST_IMG_SHAPE), ys=jnp.ones((1), dtype=jnp.int32))
svi_unsupervised.init(random.PRNGKey(0), xs=jnp.ones((1,)+MNIST_IMG_SHAPE))
svi_classify.init(random.PRNGKey(0), xs=jnp.ones((1,)+MNIST_IMG_SHAPE), ys=jnp.ones((1), dtype=jnp.int32))

print("SVI set up complete!")

SVI set up complete!


C:\Users\Matthieu Dinot\AppData\Local\Temp\ipykernel_17196\3335124818.py:2: UserWarning: Currently, SVI with Trace_ELBO loss does not support models with discrete latent variables
  svi_unsupervised.init(random.PRNGKey(0), xs=jnp.ones((1,)+MNIST_IMG_SHAPE))


In [30]:
print(state[0][1][0].keys())

dict_keys(['decoder$params', 'encoder1$params', 'encoder2$params'])


In [34]:
for batch in loader_dict["supervised"]:
    x, y = batch
    ypred = m2_vae.classify(state, x)
    print(jnp.sum(ypred==y))
    break

0


In [ ]:
tot_loss_supervised = 0.0
tot_loss_unsupervised = 0.0
tot_loss_classify = 0.0

for is_supervised, batch in loader_dict["semi_supervised"]:
    if is_supervised:
        x, y = batch
        state, loss = svi_supervised.update(state, xs=x, ys=y)
        tot_loss_supervised += loss
        state, loss = svi_classify.update(state, xs=x, ys=y)
        tot_loss_classify += loss
    else:
        x = batch
        state, loss = svi_unsupervised.update(state, xs=x)
        tot_loss_unsupervised += loss

print(tot_loss_supervised, tot_loss_unsupervised, tot_loss_classify)